# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [99]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression 
from imblearn.over_sampling import RandomOverSampler 
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import ClusterCentroids 
from imblearn.combine import SMOTEENN 
from imblearn.ensemble import BalancedRandomForestClassifier   
from imblearn.ensemble import EasyEnsembleClassifier

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('./LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns=['loan_status'])
X = pd.get_dummies(X, drop_first=False)

# Create our target
y = df['loan_status']

In [7]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
#Here Begins classification w/ random oversampler

In [21]:
ros = RandomOverSampler() 
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [22]:
y_resampled.value_counts()

low_risk     51352
high_risk    51352
Name: loan_status, dtype: int64

In [23]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [24]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'high_risk',
       'low_risk'], dtype=object)

In [26]:
balanced_accuracy_score(y_test,y_predict)

0.620091373871424

In [27]:
confusion_matrix(y_test,y_predict)

array([[   52,    35],
       [ 6120, 10998]])

In [29]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.64      0.02      0.62      0.38        87
   low_risk       1.00      0.64      0.60      0.78      0.62      0.39     17118

avg / total       0.99      0.64      0.60      0.78      0.62      0.39     17205



In [ ]:
#Here beigns classification w/ SMOTE

In [40]:
smo = SMOTE() 
X_resampled, y_resampled = smo.fit_resample(X_train, y_train)

In [41]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [42]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'high_risk', 'high_risk', ..., 'low_risk', 'high_risk',
       'low_risk'], dtype=object)

In [43]:
balanced_accuracy_score(y_test, y_predict)

0.6535437591404087

In [44]:
confusion_matrix(y_test,y_predict)

array([[   55,    32],
       [ 5565, 11553]])

In [45]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.67      0.02      0.65      0.42        87
   low_risk       1.00      0.67      0.63      0.81      0.65      0.43     17118

avg / total       0.99      0.67      0.63      0.80      0.65      0.43     17205



In [ ]:
#Here begins classification w/ Cluster Centroids

In [50]:
cc = ClusterCentroids()
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [51]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [52]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'high_risk', 'low_risk', ..., 'high_risk', 'high_risk',
       'low_risk'], dtype=object)

In [53]:
balanced_accuracy_score(y_test, y_predict)

0.5726206735398511

In [54]:
confusion_matrix(y_test,y_predict)

array([[  57,   30],
       [8729, 8389]])

In [55]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.49      0.01      0.57      0.33        87
   low_risk       1.00      0.49      0.66      0.66      0.57      0.32     17118

avg / total       0.99      0.49      0.65      0.65      0.57      0.32     17205



In [ ]:
#Here Begins classification w/ SMOTEENN

In [63]:
smoteenn = SMOTEENN()
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)

In [64]:
classifier=LogisticRegression() 
classifier.fit(X_resampled, y_resampled)

LogisticRegression()

In [65]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'high_risk',
       'low_risk'], dtype=object)

In [66]:
balanced_accuracy_score(y_test, y_predict)

0.6347260999713953

In [67]:
confusion_matrix(y_test,y_predict)

array([[   56,    31],
       [ 6406, 10712]])

In [68]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.63      0.02      0.63      0.40        87
   low_risk       1.00      0.63      0.64      0.77      0.63      0.40     17118

avg / total       0.99      0.63      0.64      0.77      0.63      0.40     17205



In [ ]:
#Here beings classification with BalancedRandomForestClassifier

In [123]:
classifier = BalancedRandomForestClassifier()

In [124]:
classifier.fit(X_train, y_train)

BalancedRandomForestClassifier()

In [125]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [126]:
balanced_accuracy_score(y_test, y_predict)

0.7781131779010599

In [127]:
confusion_matrix(y_test,y_predict)

array([[   56,    31],
       [ 1497, 15621]])

In [128]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.64      0.91      0.07      0.77      0.57        87
   low_risk       1.00      0.91      0.64      0.95      0.77      0.60     17118

avg / total       0.99      0.91      0.65      0.95      0.77      0.60     17205



In [139]:
df = pd.DataFrame({"name":classifier.feature_names_in_, "score": classifier.feature_importances_})
df=df.sort_values('score', ascending=False) 
print(df.to_string())

                                   name     score
16                        total_rec_int  0.071288
15                      total_rec_prncp  0.067028
14                      total_pymnt_inv  0.064890
20                      last_pymnt_amnt  0.055789
13                          total_pymnt  0.050367
1                              int_rate  0.025499
84                     issue_d_Jan-2019  0.020520
4                                   dti  0.019688
3                            annual_inc  0.019347
2                           installment  0.017163
12                        out_prncp_inv  0.016682
42                          avg_cur_bal  0.015992
53                mths_since_recent_inq  0.015927
74                       total_bc_limit  0.015439
25                          tot_cur_bal  0.014930
11                            out_prncp  0.014721
72                      tot_hi_cred_lim  0.014252
43                       bc_open_to_buy  0.013986
9                             revol_bal  0.013871


In [ ]:
# Here beings classification with EasyEnsembleClassifier

In [114]:
classifier = EasyEnsembleClassifier(n_estimators = 100)

In [115]:
classifier.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100)

In [116]:
y_predict = classifier.predict(X_test) 
y_predict

array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [117]:
balanced_accuracy_score(y_test, y_predict)

0.9010730789529875

In [118]:
confusion_matrix(y_test,y_predict)

array([[   74,    13],
       [  829, 16289]])

In [119]:
print(classification_report_imbalanced(y_test,y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.85      0.95      0.15      0.90      0.80        87
   low_risk       1.00      0.95      0.85      0.97      0.90      0.82     17118

avg / total       0.99      0.95      0.85      0.97      0.90      0.82     17205

